In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import multiprocessing as mp
from multiprocessing import Process, Pool
import pycorenlp
import pandas as pd
from pandas.io.json import json_normalize

# Majority of code for this assignment located in pdfscrape and cityscrape folders
from pdfscrape import pdf_pipeline_ns
from cityscrape import scrape

In [6]:
# nlp = pycorenlp.StanfordCoreNLP("http://ec2-18-234-241-82.compute-1.amazonaws.com:9000")
nlp = pycorenlp.StanfordCoreNLP("http://localhost:9000")

In [11]:
tovisit_q = mp.Queue()
writeto_q = mp.Queue()
faildrd_q = mp.Queue()
pdflink_q = mp.Queue()
visited_qs = mp.Queue()
visited_qs.put(set())
tovisit_qs = mp.Queue()
tovisit_qs.put(set())

In [12]:
w = scrape.WebScrape('http://cityofchicago.org',
                     tovisit_q,
                     writeto_q,
                     faildrd_q,
                     pdflink_q,
                     visited_qs,
                     tovisit_qs,
                     lmt_doma='www.cityofchicago.org')

arr = (pdflink_q, 10, 5, 5, 30, './scrape1.csv', 'temp1.pdf',False, nlp)
arr2 = (pdflink_q, 10, 5, 5, 60, './scrape2.csv', 'temp2.pdf', True, nlp)

In [14]:
OMIT_WORDS = ['department', 'spec', 'council', 'please', '311','phone','press','release']

p1 = Process(name="Process1",
             target=w.scrape,
             args=(100,OMIT_WORDS))

p2 = Process(name="Process PDFs",
             target=pdf_pipeline_ns.scrape_pdfs,
             args=(*arr,))

p3 = Process(name="Process3",
            target=pdf_pipeline_ns.scrape_pdfs,
             args=(*arr2,))


p1.start()
p2.start()
p3.start()


p1.join()
p2.join()
p3.join()

http://cityofchicago.org 215
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dhr/supp_info/BID_-_Create_User_Name_Email_Profile_REQs_4-09.pdf
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dhr/supp_info/Candidate_Experience__v14_3-10.pdf
https://www.cityofchicago.org/content/city/en/depts/dhr/provdrs/emp/svcs/apply_for_bid_opportunities-forcityofchicagoemployeesandbargaini.html 376
https://www.cityofchicago.org/content/city/en/depts/bldgs/provdrs/e_plan/svcs/dynamicportal.html 529
https://www.cityofchicago.org/content/city/en/depts/bacp/provdrs/bus/svcs/apply_for_a_businesslicenseonline.html 765
https://www.cityofchicago.org/content/city/en/depts/other/provdrs/cha/svcs/find_housing_information.html 972
DOWNLOADED
https://www.cityofchicago.org/content/city/en/depts/dhr/provdrs/emp/svcs/internships.html 1128
DOWNLOADED
https://www.cityofchicago.org/content/city/en/depts/dhr/provdrs/emp/svcs/city_of_chicago_jobopportunities.html 1287
https://

https://www.cityofchicago.org/content/city/en/depts/bacp/provdrs/consumer/svcs/consumer_cab_complaint.html 7632
https://www.cityofchicago.org/content/city/en/depts/other/provdrs/clerk/svcs/file_a_claim.html 7742
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dhr/supp_info/Internships/Press-CommOffice-PhotographyIntern.pdf
https://www.cityofchicago.org/content/city/en/depts/bacp/provdrs/pros_adj/svcs/file_a_citizen_complaintonline.html 7866
DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dhr/supp_info/Internships/Database_Developer_Intern.pdf
DOWNLOADED
https://www.cityofchicago.org/content/city/en/depts/dps/provdrs/comp/svcs/economic_disclosurestatementseds.html 8015
https://www.cityofchicago.org/content/city/en/depts/cdot/provdrs/street/svcs/report_a_pot_holeinstreet.html 8107
https://www.cityofchicago.org/content/city/en/depts/cacc/provdrs/animal_control_andrescue/svcs/report_a_stray_animal.html 8212
https://ww

DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dps/Outreach/GuidetoBidIncentivesPrograms_110117.pdf
DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dps/WeeklyBidOpportunities/2018WeeklBidOpportunities/82718.pdf
DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dps/ContractAdministration/Specs/2018/Spec704315.pdf
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dps/ContractAdministration/Addendum/2018Addendum/Spec704315Exhibit7B.pdf
DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dps/ContractAdministration/Addendum/2018Addendum/Spec704315Exhibit7D.pdf
DOWNLOADED
DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts/dps/ContractAdministration/Specs/2018/RFPMMFRFP2018.pdf
DOWNLOADED
SUCCESS
SCRAPING PDF URL: https://www.cityofchicago.org/content/dam/city/depts